In [1]:
import os
import time
from io import StringIO
import subprocess
import plotnine
import pandas as pd
import numpy as np

In [2]:
base_dir="/vortexfs1/omics/alexander/akrinos/2022-euk-diversity/output/tara-eukulele-mapping/EUKulele_assembly_mapping"
eukulele_paths=os.listdir(os.path.join(base_dir))

In [3]:
coverm_dir="/vortexfs1/scratch/akrinos/2022-euk-diversity-phaeo-contigs"
all_phaeo_seqs = pd.DataFrame()
for curr_path in eukulele_paths:
    if not os.path.isfile(os.path.join(coverm_dir,curr_path,"coverm","phaeo_abund.out")):
        continue
    curr_all_combo = pd.DataFrame()
    if os.path.getsize(os.path.join(coverm_dir,curr_path,"coverm","phaeo_abund.out")) == 0:
        continue
    step1=pd.read_csv(os.path.join(coverm_dir,curr_path,"coverm","phaeo_abund.out"),
        sep="\t")
    step1["TotalAll"] = step1.loc[:,[("phaeo_contigs" in curr)&("TPM" in curr) \
                                     for curr in step1.columns]].sum(axis=1)
    for curr_path_2 in os.listdir(os.path.join(base_dir,curr_path)):
        if not (os.path.isdir(os.path.join(base_dir, curr_path, curr_path_2, "taxonomy_estimation"))):
            continue
        tax_est_file = os.path.join(base_dir, curr_path, curr_path_2, "taxonomy_estimation",
                                    "proteins-estimated-taxonomy.out")
        output = subprocess.check_output(" ".join(["cat", tax_est_file, "| { grep Phaeocystis || true; }"]),
                                          shell=True).decode("utf-8").split("\n")
        TESTDATA = StringIO("\n".join(output))
        df = pd.read_csv(TESTDATA, sep="\t", header=None,names=["transcript_name","classification_level",
                                                        "full_classification","classification",
                                                        "max_pid","ambiguous","counts"])
        df["contig_name"] = ["_".join(curr.split("_")[0:2]) for curr in df["transcript_name"]]
        df = df.merge(step1.loc[:,["Contig","TotalAll"]],left_on="contig_name",
                      right_on="Contig",how="left")
        tara_sample=curr_path
        phaeo_curr=curr_path_2
        df["PhaeoDB"]=phaeo_curr
        df["TaraSample"]=curr_path
        all_phaeo_seqs = pd.concat([all_phaeo_seqs,df])

In [4]:
counted_by_samp = all_phaeo_seqs.drop_duplicates(subset=["transcript_name",
                                                         "TaraSample"]).\
                    groupby("TaraSample")["classification"].count().reset_index().\
    rename({"classification":"TotalBySamp"},axis="columns")

In [5]:
grouped_counted_seqs = all_phaeo_seqs.loc[[(curr in ["phaeo_all","free_phaeo","colonies_phaeo"]) \
                    for curr in all_phaeo_seqs.PhaeoDB],:].groupby(["PhaeoDB",
                                                                    "TaraSample"])\
    ["classification"].count().reset_index()
grouped_counted_seqs["ShortCode"] = [curr.split("-")[0] for curr in grouped_counted_seqs.TaraSample]
grouped_counted_seqs["Depth"] = [curr.split("-")[2] for curr in grouped_counted_seqs.TaraSample]
grouped_counted_seqs["SizeFract"] = ["Small" if "0.8-5" in curr else "Mid" if "5-20" in curr\
                                     else "Large" if "20-" in curr else "Largest" \
                                     for curr in grouped_counted_seqs.TaraSample]

In [6]:
grouped_counted_seqs_abund = all_phaeo_seqs.loc[[(curr in ["phaeo_all","free_phaeo","colonies_phaeo"]) \
                    for curr in all_phaeo_seqs.PhaeoDB],:].fillna(0).groupby(["PhaeoDB",
                                                                    "TaraSample"])\
    ["TotalAll"].sum().reset_index()
grouped_counted_seqs_abund["ShortCode"] = [curr.split("-")[0] for curr in grouped_counted_seqs_abund.TaraSample]
grouped_counted_seqs_abund["Depth"] = [curr.split("-")[2] for curr in grouped_counted_seqs_abund.TaraSample]
grouped_counted_seqs_abund["SizeFract"] = ["Small" if "0.8-5" in curr else "Mid" if "5-20" in curr\
                                     else "Large" if "20-" in curr else "Largest" \
                                     for curr in grouped_counted_seqs_abund.TaraSample]

In [7]:
grouped_counted_seqs.to_csv("short_code_abund_9July2023.csv")

In [8]:
grouped_counted_seqs_abund.to_csv("short_code_abund_mean_cov_9July2023.csv")